In [21]:
from cobra.io import load_matlab_model, save_matlab_model
Recon3DModel_301 = load_matlab_model('C:/Users/prins/git/COBRA_project_SP/data/Recon3D_model/Recon3DModel_301_patch.mat') # load Recon3D model (this takes really long!!)

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [6]:
from cobra import Configuration
config = Configuration()
config.solver = "glpk"
config

Attribute,Description,Value
solver,Mathematical optimization solver,glpk
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,7
cache_directory,Path for the model cache,C:\Users\prins\AppData\Local\opencobra\cobrapy\Cache
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


In [7]:
from corda import reaction_confidence
from cobra import Reaction

gene_conf = {"gene1": 1, "gene2": 3, "gene4": -1,"gene5":2} # missing entries are automatically assigned zeroes
rxn = Reaction("my_reaction")
rxn.gene_reaction_rule = "gene1 and gene2 or (gene3 and gene4) or gene5 and gene1"

reaction_confidence(rxn, gene_conf)

1

In [95]:
from corda import test_model

mod = test_model()
len(mod.reactions)

60

In [96]:
mod

Name,cemet
Memory address,24ac70bc220
Number of metabolites,43
Number of reactions,60
Number of genes,0
Number of groups,0
Objective expression,1.0*r60 - 1.0*r60_reverse_c3303
Compartments,cellular interior


In [10]:
mod.reactions[59].reaction

'0.39253 3pg + 20.7045 atp + 0.15446 cit + 0.38587 glu + 0.35261 oaa + 0.053446 prpp + 0.50563 pyr --> 20.6508 adp + 20.6508 pi'

In [97]:
conf = {}
for r in mod.reactions: conf[r.id] = -1
conf["r60"] = 3

In [98]:
from corda import CORDA

opt = CORDA(mod, conf)
opt.build()
print(opt)

build status: reconstruction complete
Inc. reactions: 33/60
 - unclear: 0/0
 - exclude: 32/59
 - low and medium: 0/0
 - high: 1/1



In [109]:
([opt.model.reactions.get_by_id(k).reaction for k, used in opt.included.items() if used])

['fdp <=> dhap + g3p',
 'g3p + nad + pi <=> 13dpg + nadh',
 'atp + f6p --> adp + fdp',
 'g6p <=> f6p',
 '3pg + atp <=> 13dpg + adp',
 'g1p <=> g6p',
 'dhap <=> g3p',
 'lac_L + nad <=> nadh + pyr',
 'atp + r5p <=> amp + prpp',
 'ru5p_D <=> xu5p_D',
 'r5p <=> ru5p_D',
 'g3p + s7p <=> e4p + f6p',
 'r5p + xu5p_D <=> g3p + s7p',
 'e4p + xu5p_D <=> f6p + g3p',
 'cit <=> icit',
 'icit + nad <=> akg + nadh',
 'icit + nadp <=> akg + nadph',
 'mal_L + nad <=> nadh + oaa',
 'mal_L + nad <=> nadh + pyr',
 'mal_L + nadp <=> nadph + pyr',
 'atp + pyr --> adp + oaa + pi',
 'gln <=> glu',
 'glu + nad <=> akg + nadh',
 'glu + nadp <=> akg + nadph',
 'adp + pi --> atp',
 'nadh --> nad',
 ' --> g1p',
 ' <=> gln',
 ' --> adp',
 ' <=> amp',
 ' <=> pi',
 'lac_L <=> ',
 '0.39253 3pg + 20.7045 atp + 0.15446 cit + 0.38587 glu + 0.35261 oaa + 0.053446 prpp + 0.50563 pyr --> 20.6508 adp + 20.6508 pi']

In [48]:
Recon3DModel_301.metabolites[1].name
list(Recon3DModel_301.metabolites.get_by_id("atp[l]").reactions)[1]

Reaction identifier,r0859
Name,Postulated Transport Reaction
Memory address,0x249da29ca30
Stoichiometry,adp[l] + atp[c] --> adp[c] + atp[l] Adenosine Diphosphate + Adenosine Triphosphate --> Adenosine Diphosphate + Adenosine Triphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [52]:
Recon3DModel_301.reactions.get_by_id("1MNCAMti")

Reaction identifier,1MNCAMti
Name,N1-Methylnicotinamide Transport
Memory address,0x24933c42380
Stoichiometry,1mncam[c] + atp[c] + h2o[c] --> 1mncam[e] + adp[c] + h[c] + pi[c] 1-Methylnicotinamide + Adenosine Triphosphate + Water --> 1-Methylnicotinamide + Adenosine Diphosphate + Proton + Orthophosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


reconstruct the smallest model that still contains all reactions involving ATP (confidence level 3)

In [ ]:
Recon3DModel_301.metabolites.get_by_id("atp[c]")

In [53]:
conf = {}
for r in Recon3DModel_301.reactions: conf[r.id] = -1
for m in Recon3DModel_301.metabolites:
    if m.name == 'Adenosine Triphosphate':
        for r in m.reactions: conf[r.id] = 3

In [54]:
opt = CORDA(Recon3DModel_301, conf)
opt.build()
print(opt)

build status: reconstruction complete
Inc. reactions: 3765/10601
 - unclear: 0/0
 - exclude: 2837/9673
 - low and medium: 0/0
 - high: 928/928



In [94]:
dir(opt)
opt.cobra_model

<bound method CORDA.cobra_model of <corda.corda.CORDA object at 0x000002498EDAF640>>

In [71]:
Recon3DModel_301

Name,model
Memory address,249332dce80
Number of metabolites,5835
Number of reactions,10601
Number of genes,2248
Number of groups,104
Objective expression,1.0*biomass_maintenance - 1.0*biomass_maintenance_reverse_95d2f
Compartments,"c, l, m, r, e, x, n, g, i"


In [73]:
from corda import benchmark
benchmark(Recon3DModel_301)

Running setup for model `model`.

Running CORDA setup...

✔ [112 s]

Running CORDA build...

✔ [21.5 s]

Running validation on reduced model...

✔ [241 s]

In [69]:
from cobra.io import save_json_model
save_json_model(opt.model, "Recon3DModel_301_ATPreactions.json")
save_json_model(Recon3DModel_301, "Recon3DModel_301.json")

In [70]:
([opt.model.reactions.get_by_id(k).reaction for k, used in opt.included.items() if used])

['10fthf5glu[c] --> 10fthf5glu[l]',
 '10fthf5glu[m] --> 10fthf5glu[c]',
 '10fthf6glu[c] --> 10fthf6glu[l]',
 '10fthf6glu[m] --> 10fthf6glu[c]',
 '10fthf7glu[c] --> 10fthf7glu[l]',
 '10fthf7glu[m] --> 10fthf7glu[c]',
 '10fthf[c] <=> 10fthf[l]',
 '10fthf[c] <=> 10fthf[m]',
 '1mncam[c] + atp[c] + h2o[c] --> 1mncam[e] + adp[c] + h[c] + pi[c]',
 '2dr1p[c] + h2o[c] --> drib[c] + pi[c]',
 '2hb[e] + h[e] <=> 2hb[c] + h[c]',
 '34dhphe[e] <=> 34dhphe[c]',
 'coa[m] + coucoa[m] + h2o[m] + nad[m] --> 4hbzcoa[m] + accoa[m] + h[m] + nadh[m]',
 '4hbzcoa[m] + h2o[m] --> 4hbz[m] + coa[m] + h[m]',
 '4mop[c] + h[c] --> 4mop[m] + h[m]',
 '4mptnl[c] <=> 4mptnl[e]',
 '4mptnl[c] <=> 4mptnl[m]',
 '4pyrdx[c] + atp[c] + h2o[c] --> 4pyrdx[e] + adp[c] + h[c] + pi[c]',
 '5adtststeroneglc[c] + atp[c] + h2o[c] --> 5adtststeroneglc[e] + adp[c] + h[c] + pi[c]',
 '5adtststeroneglc[c] <=> 5adtststeroneglc[r]',
 '5adtststerones[c] <=> 5adtststerones[e]',
 '5adtststerone[e] <=> 5adtststerone[c]',
 '5adtststerone[r] <=> 5ad